In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd
import shutil
import os
import numpy as np

# example of model file names : 
# model_imagesPerEntity_1647_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras
# model_imagesPerEntity_2481_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras

def copy_models_to_kaggle(model_paths):
    os.makedirs('/kaggle/working/models', exist_ok=True)
    new_paths = []
    for model_path in model_paths:
        model_file_name = model_path.split("/")[-1]        
        new_model_path = '/kaggle/working/models/' + model_file_name
        new_paths.append(new_model_path)
        if os.path.exists(new_model_path):
            print(f"model already exists at {new_model_path}")
        shutil.copy(model_path, new_model_path)
    return new_paths
 

batch_size = 32
test_dataset_paths = [
    '/kaggle/input/clean-2500-letters-only',
    '/kaggle/input/clean-1800-letters-only'
]
models_paths = [
    r'/kaggle/input/1800-model/keras/16/1/model_imagesPerEntity_1647_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/1800-model/keras/32/1/model_imagesPerEntity_1647_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/1800-model/keras/64/1/model_imagesPerEntity_1647_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/1800-model/keras/128/2/model_imagesPerEntity_1647_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/1800-model/keras/256/1/model_imagesPerEntity_1647_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/1800-model/keras/300/1/model_imagesPerEntity_1647_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras',

    r'/kaggle/input/2500-model/keras/model-grayscale-16/1/model_imagesPerEntity_2481_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/2500-model/keras/model-grayscale-32/1/model_imagesPerEntity_2481_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/2500-model/keras/model-grayscale-64/1/model_imagesPerEntity_2481_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/2500-model/keras/model-grayscale-128/1/model_imagesPerEntity_2481_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/2500-model/keras/model-grayscale-256/1/model_imagesPerEntity_2481_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras',
    r'/kaggle/input/2500-model/keras/model-grayscale-300/1/model_imagesPerEntity_2481_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras'

]

print("copying models to kaggle...")
copied_models_paths = copy_models_to_kaggle(models_paths)
print(f"loading models in RAM...")
model_path_to_model = {}
for model_path in copied_models_paths:
    print(f"loading model in : {model_path} ...")
    model = load_model(model_path)
    model_path_to_model[model_path] = model

all_datasets_accuracies = {}
for dataset_path in test_dataset_paths:
    dataset_name  = dataset_path.split("/")[-1]
    print(f"Testing dataset : {dataset_path} ...")
    dataset_test_accuracies = {}
    for model_path in copied_models_paths:
        model_file_name_parts = model_path.split("/")[-1].split(".")[0].split("_")
        trained_data_size = model_file_name_parts[2]
        img_size = model_file_name_parts[4]
        current_model_name =  f"{trained_data_size}_dataset_{img_size}_img_size"
        model = model_path_to_model[model_path]
        print(f"trained_data_size : {trained_data_size}, img_size : {img_size}, model_name : {current_model_name} ...")
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            dataset_path,
            target_size=(int(img_size), int(img_size)),
            batch_size=batch_size,
            class_mode='categorical',
            color_mode='grayscale',
            shuffle=False)
        print()

        # Predict the output
        steps = int(np.ceil(test_generator.samples / test_generator.batch_size))
        predictions = model.predict(test_generator, steps=steps)
        predictions = predictions.argmax(axis=-1)                    
        true_classes = test_generator.classes
        accuracy =  round(accuracy_score(true_classes, predictions), 3)
        dataset_test_accuracies[current_model_name] = accuracy
        print(f"Accuracy of model {current_model_name} on dataset {dataset_name} : {accuracy}")
    all_datasets_accuracies[dataset_name] = dataset_test_accuracies

print(f"all datasets accuracies : {all_datasets_accuracies}")
accuracies_df = pd.DataFrame(all_datasets_accuracies).transpose()
accuracies_df = accuracies_df.style.background_gradient(cmap='tab20_r').set_table_styles([dict(selector="th", props=[("text-align", "center")])])
display(accuracies_df)


copying models to kaggle...
model already exists at /kaggle/working/models/model_imagesPerEntity_1647_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle/working/models/model_imagesPerEntity_1647_imgSize_32_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle/working/models/model_imagesPerEntity_1647_imgSize_64_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle/working/models/model_imagesPerEntity_1647_imgSize_128_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle/working/models/model_imagesPerEntity_1647_imgSize_256_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle/working/models/model_imagesPerEntity_1647_imgSize_300_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle/working/models/model_imagesPerEntity_2481_imgSize_16_dataPercentage_100_batchSize_32_epochs_100.keras
model already exists at /kaggle

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2171/2171 ━━━━━━━━━━━━━━━━━━━━ 68s 31ms/step
Accuracy of model 1647_dataset_32_img_size on dataset clean-2500-letters-only : 0.194
trained_data_size : 1647, img_size : 64, model_name : 1647_dataset_64_img_size ...
Found 69468 images belonging to 28 classes.



/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2171/2171 ━━━━━━━━━━━━━━━━━━━━ 61s 28ms/step
Accuracy of model 1647_dataset_64_img_size on dataset clean-2500-letters-only : 0.092
trained_data_size : 1647, img_size : 128, model_name : 1647_dataset_128_img_size ...
Found 69468 images belonging to 28 classes.



...

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1442/1442 ━━━━━━━━━━━━━━━━━━━━ 58s 40ms/step
Accuracy of model 2481_dataset_256_img_size on dataset clean-1800-letters-only : 0.098
trained_data_size : 2481, img_size : 300, model_name : 2481_dataset_300_img_size ...
Found 46130 images belonging to 28 classes.

   4/1442 ━━━━━━━━━━━━━━━━━━━━ 1:03 44ms/step

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1442/1442 ━━━━━━━━━━━━━━━━━━━━ 63s 44ms/step
Accuracy of model 2481_dataset_300_img_size on dataset clean-1800-letters-only : 0.149
all datasets accuracies : {'clean-2500-letters-only': {'1647_dataset_16_img_size': 0.07, '1647_dataset_32_img_size': 0.194, '1647_dataset_64_img_size': 0.092, '1647_dataset_128_img_size': 0.178, '1647_dataset_256_img_size': 0.111, '1647_dataset_300_img_size': 0.134, '2481_dataset_16_img_size': 0.954, '2481_dataset_32_img_size': 0.995, '2481_dataset_64_img_size': 0.992, '2481_dataset_128_img_size': 0.991, '2481_dataset_256_img_size': 0.936, '2481_dataset_300_img_size': 0.973}, 'clean-1800-letters-only': {'1647_dataset_16_img_size': 0.141, '1647_dataset_32_img_size': 0.549, '1647_dataset_64_img_size': 0.898, '1647_dataset_128_img_size': 0.631, '1647_dataset_256_img_size': 0.533, '1647_dataset_300_img_size': 0.67, '2481_dataset_16_img_size': 0.087, '2481_dataset_32_img_size': 0.147, '2481_dataset_64_img_size': 0.129, '2481_dataset_128_img_size': 0.152, '2481_

,1647_dataset_16_img_size,1647_dataset_32_img_size,1647_dataset_64_img_size,1647_dataset_128_img_size,1647_dataset_256_img_size,1647_dataset_300_img_size,2481_dataset_16_img_size,2481_dataset_32_img_size,2481_dataset_64_img_size,2481_dataset_128_img_size,2481_dataset_256_img_size,2481_dataset_300_img_size
clean-2500-letters-only,0.070000,0.194000,0.092000,0.178000,0.111000,0.134000,0.954000,0.995000,0.992000,0.991000,0.936000,0.973000
clean-1800-letters-only,0.141000,0.549000,0.898000,0.631000,0.533000,0.670000,0.087000,0.147000,0.129000,0.152000,0.098000,0.149000
